# ML Notebook 2 - Random Forest & XGBoost 


Notebook is mostly just me tinkering with these two classifiers, despite them not performing well in my initial test of various classifiers.

In [1]:
import os
import sys
sys.path.append('../')
import glob2 as glob
import json
import requests
import numpy as np
import pandas as pd
#import seaborn as sns
import matplotlib.pyplot as plt
#import plotly.express as px

pd.options.display.max_columns = 200
pd.options.display.max_rows = 200
%matplotlib inline

import warnings; warnings.simplefilter('ignore')

In [2]:
# Sklearn stuff
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.model_selection import ShuffleSplit, KFold

In [3]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, roc_auc_score

## Random Forest 

We're gonna test whether scaling makes much of a difference.

1. Run random forest with just hot encoding. 
2. Run random forest with hot encoding and minmax scaler

In [4]:
SEED = 17

In [11]:
# Import Training and Test Data  
train_features = pd.read_csv('data2/dpc_train_features.csv', index_col='match_id')
train_targets = pd.read_csv('data2/dpc_train_target.csv', index_col='match_id')
test_features = pd.read_csv('data2/dpc_test_features.csv', index_col='match_id')
test_targets = pd.read_csv('data2/dpc_test_target.csv', index_col='match_id')

In [6]:
X_rremainder = train_features.copy()
X_rtest = test_features.copy()
y_rremainder = train_targets.radiant_win.values.copy()
y_rtest = test_targets.radiant_win.values.copy()

In [7]:
train_features.head()

,1is_pick,1team,1hero_id,2is_pick,2team,2hero_id,3is_pick,3team,3hero_id,4is_pick,4team,4hero_id,5is_pick,5team,5hero_id,6is_pick,6team,6hero_id,7is_pick,7team,7hero_id,8is_pick,8team,8hero_id,9is_pick,9team,9hero_id,10is_pick,10team,10hero_id,11is_pick,11team,11hero_id,12is_pick,12team,12hero_id,13is_pick,13team,13hero_id,14is_pick,14team,14hero_id,15is_pick,15team,15hero_id,16is_pick,16team,16hero_id,17is_pick,17team,17hero_id,18is_pick,18team,18hero_id,19is_pick,19team,19hero_id,20is_pick,20team,20hero_id,21is_pick,21team,21hero_id,22is_pick,22team,22hero_id
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5115031896,0,0,25,0,1,66,0,0,60,0,1,90,0,0,7,0,1,96,1,0,85,1,1,102,1,1,6,1,0,52,0,0,13,0,1,19,0,0,46,0,1,98,1,1,86,1,0,20,1,1,72,1,0,77,0,1,23,0,0,112,1,0,129,1,1,2
4865407245,0,1,71,0,0,18,0,1,55,0,0,66,0,1,129,0,0,96,1,1,37,1,0,121,1,0,103,1,1,100,0,1,106,0,0,104,0,1,16,0,0,76,1,0,65,1,1,8,1,0,46,1,1,57,0,0,114,0,1,10,1,1,17,1,0,42
4889341660,0,1,105,0,0,107,0,1,121,0,0,80,0,1,86,0,0,66,1,1,57,1,0,18,1,0,111,1,1,95,0,1,96,0,0,11,0,1,55,0,0,52,1,0,79,1,1,27,1,0,2,1,1,85,0,0,17,0,1,35,1,1,76,1,0,43
5050335216,0,0,84,0,1,90,0,0,19,0,1,79,0,0,60,0,1,23,1,0,103,1,1,96,1,1,86,1,0,106,0,0,61,0,1,10,0,0,93,0,1,41,1,1,37,1,0,9,1,1,72,1,0,76,0,1,16,0,0,39,1,0,108,1,1,59
4860032461,0,1,111,0,0,18,0,1,69,0,0,55,0,1,65,0,0,8,1,1,66,1,0,121,1,0,16,1,1,107,0,1,42,0,0,46,0,1,10,0,0,54,1,0,37,1,1,96,1,0,106,1,1,76,0,0,95,0,1,82,1,1,1,1,0,73


In [8]:
test_features.head()

,1is_pick,1team,1hero_id,2is_pick,2team,2hero_id,3is_pick,3team,3hero_id,4is_pick,4team,4hero_id,5is_pick,5team,5hero_id,6is_pick,6team,6hero_id,7is_pick,7team,7hero_id,8is_pick,8team,8hero_id,9is_pick,9team,9hero_id,10is_pick,10team,10hero_id,11is_pick,11team,11hero_id,12is_pick,12team,12hero_id,13is_pick,13team,13hero_id,14is_pick,14team,14hero_id,15is_pick,15team,15hero_id,16is_pick,16team,16hero_id,17is_pick,17team,17hero_id,18is_pick,18team,18hero_id,19is_pick,19team,19hero_id,20is_pick,20team,20hero_id,21is_pick,21team,21hero_id,22is_pick,22team,22hero_id
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4870161807,0,1,66,0,0,97,0,1,91,0,0,71,0,1,111,0,0,55,1,1,121,1,0,18,1,0,37,1,1,65,0,1,57,0,0,106,0,1,76,0,0,93,1,0,88,1,1,10,1,0,104,1,1,69,0,0,114,0,1,17,1,1,79,1,0,52
5103391298,0,0,60,0,1,102,0,0,7,0,1,41,0,0,90,0,1,66,1,0,25,1,1,9,1,1,19,1,0,23,0,0,15,0,1,54,0,0,72,0,1,46,1,1,31,1,0,85,1,1,8,1,0,76,0,1,6,0,0,35,1,0,4,1,1,22
5062411190,0,0,9,0,1,66,0,0,52,0,1,60,0,0,97,0,1,23,1,0,25,1,1,5,1,1,41,1,0,120,0,0,74,0,1,72,0,0,86,0,1,113,1,1,101,1,0,111,1,1,16,1,0,100,0,1,1,0,0,34,1,0,81,1,1,49
4971396775,0,1,72,0,0,66,0,1,52,0,0,58,0,1,73,0,0,68,1,1,9,1,0,84,1,0,79,1,1,103,0,1,25,0,0,93,0,1,106,0,0,10,1,0,29,1,1,8,1,0,54,1,1,15,0,0,97,0,1,36,1,1,61,1,0,23
5059615080,0,1,97,0,0,41,0,1,9,0,0,102,0,1,66,0,0,85,1,1,23,1,0,57,1,0,121,1,1,104,0,1,98,0,0,43,0,1,76,0,0,101,1,0,5,1,1,30,1,0,93,1,1,72,0,0,91,0,1,106,1,1,90,1,0,54


In [9]:
train_features.shape, train_targets.shape, test_features.shape, test_targets.shape

((1045, 66), (1045, 2), (117, 66), (117, 2))

Let's try applying get dummies on training and test features seperately

HMMMMM well that doesn't look right. It seems that some heroes don't show up in certain pick orders in the test set (my best guess). 

Guess this means I should get dummies on the entire feature set combined

In [12]:
# Combining training and test features for convenience so I can hot encode both at same time
all_features = pd.concat([train_features, test_features])

# Index to split the training and test data sets
idx_split = train_features.shape[0]

In [13]:
for i in range(1,23):
        all_features = pd.get_dummies(all_features, columns = [f'{i}hero_id'])

print(all_features.shape)

(1162, 2088)


Now that looks more reasonable...

In [1]:
# Encoded train/test features
# Try this once without scaling 

#X_remainder = all_features[:idx_split].astype('int')
#X_test = all_features[idx_split:].astype('int')

#y_remainder = train_targets.radiant_win.astype('int')
#y_test = test_targets.radiant_win.astype('int')

In [14]:
# Encoded train/test features

X_remainder = all_features[:idx_split].values
X_test = all_features[idx_split:].values

y_remainder = train_targets.radiant_win.values
y_test = test_targets.radiant_win.values

In [15]:
X_remainder.shape, X_test.shape, y_remainder.shape, y_test.shape

((1045, 2088), (117, 2088), (1045,), (117,))

Quick eyeball check to see if the merge-split mixed anything up

In [58]:
X_remainder.head()

,1is_pick,1team,2is_pick,2team,3is_pick,3team,4is_pick,4team,5is_pick,5team,6is_pick,6team,7is_pick,7team,8is_pick,8team,9is_pick,9team,10is_pick,10team,11is_pick,11team,12is_pick,12team,13is_pick,13team,14is_pick,14team,15is_pick,15team,16is_pick,16team,17is_pick,17team,18is_pick,18team,19is_pick,19team,20is_pick,20team,21is_pick,21team,22is_pick,22team,1hero_id_2,1hero_id_3,1hero_id_4,1hero_id_6,1hero_id_7,1hero_id_9,1hero_id_10,1hero_id_16,1hero_id_18,1hero_id_19,1hero_id_23,1hero_id_25,1hero_id_27,1hero_id_29,1hero_id_31,1hero_id_33,1hero_id_37,1hero_id_38,1hero_id_40,1hero_id_41,1hero_id_42,1hero_id_43,1hero_id_46,1hero_id_52,1hero_id_53,1hero_id_55,1hero_id_57,1hero_id_58,1hero_id_59,1hero_id_60,1hero_id_61,1hero_id_62,1hero_id_65,1hero_id_66,1hero_id_68,1hero_id_69,1hero_id_71,1hero_id_72,1hero_id_73,1hero_id_74,1hero_id_75,1hero_id_76,1hero_id_79,1hero_id_80,1hero_id_81,1hero_id_82,1hero_id_83,1hero_id_84,1hero_id_85,1hero_id_86,1hero_id_87,1hero_id_88,1hero_id_89,1hero_id_90,1hero_id_91,1hero_id_93,...,22hero_id_1,22hero_id_2,22hero_id_4,22hero_id_5,22hero_id_6,22hero_id_7,22hero_id_8,22hero_id_10,22hero_id_11,22hero_id_12,22hero_id_13,22hero_id_14,22hero_id_15,22hero_id_16,22hero_id_17,22hero_id_18,22hero_id_19,22hero_id_21,22hero_id_22,22hero_id_23,22hero_id_25,22hero_id_26,22hero_id_27,22hero_id_28,22hero_id_29,22hero_id_32,22hero_id_33,22hero_id_34,22hero_id_35,22hero_id_36,22hero_id_37,22hero_id_38,22hero_id_39,22hero_id_40,22hero_id_41,22hero_id_42,22hero_id_43,22hero_id_44,22hero_id_45,22hero_id_46,22hero_id_47,22hero_id_48,22hero_id_49,22hero_id_50,22hero_id_51,22hero_id_52,22hero_id_53,22hero_id_54,22hero_id_55,22hero_id_56,22hero_id_57,22hero_id_58,22hero_id_59,22hero_id_60,22hero_id_61,22hero_id_62,22hero_id_63,22hero_id_65,22hero_id_67,22hero_id_68,22hero_id_70,22hero_id_71,22hero_id_72,22hero_id_73,22hero_id_74,22hero_id_75,22hero_id_76,22hero_id_77,22hero_id_78,22hero_id_79,22hero_id_80,22hero_id_81,22hero_id_82,22hero_id_84,22hero_id_85,22hero_id_86,22hero_id_88,22hero_id_89,22hero_id_92,22hero_id_93,22hero_id_94,22hero_id_95,22hero_id_96,22hero_id_97,22hero_id_98,22hero_id_99,22hero_id_101,22hero_id_102,22hero_id_104,22hero_id_106,22hero_id_107,22hero_id_108,22hero_id_109,22hero_id_110,22hero_id_112,22hero_id_113,22hero_id_114,22hero_id_120,22hero_id_121,22hero_id_129
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5115031896,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4865407245,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4889341660,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5050335216,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,

In [59]:
X_test.head()

,1is_pick,1team,2is_pick,2team,3is_pick,3team,4is_pick,4team,5is_pick,5team,6is_pick,6team,7is_pick,7team,8is_pick,8team,9is_pick,9team,10is_pick,10team,11is_pick,11team,12is_pick,12team,13is_pick,13team,14is_pick,14team,15is_pick,15team,16is_pick,16team,17is_pick,17team,18is_pick,18team,19is_pick,19team,20is_pick,20team,21is_pick,21team,22is_pick,22team,1hero_id_2,1hero_id_3,1hero_id_4,1hero_id_6,1hero_id_7,1hero_id_9,1hero_id_10,1hero_id_16,1hero_id_18,1hero_id_19,1hero_id_23,1hero_id_25,1hero_id_27,1hero_id_29,1hero_id_31,1hero_id_33,1hero_id_37,1hero_id_38,1hero_id_40,1hero_id_41,1hero_id_42,1hero_id_43,1hero_id_46,1hero_id_52,1hero_id_53,1hero_id_55,1hero_id_57,1hero_id_58,1hero_id_59,1hero_id_60,1hero_id_61,1hero_id_62,1hero_id_65,1hero_id_66,1hero_id_68,1hero_id_69,1hero_id_71,1hero_id_72,1hero_id_73,1hero_id_74,1hero_id_75,1hero_id_76,1hero_id_79,1hero_id_80,1hero_id_81,1hero_id_82,1hero_id_83,1hero_id_84,1hero_id_85,1hero_id_86,1hero_id_87,1hero_id_88,1hero_id_89,1hero_id_90,1hero_id_91,1hero_id_93,...,22hero_id_1,22hero_id_2,22hero_id_4,22hero_id_5,22hero_id_6,22hero_id_7,22hero_id_8,22hero_id_10,22hero_id_11,22hero_id_12,22hero_id_13,22hero_id_14,22hero_id_15,22hero_id_16,22hero_id_17,22hero_id_18,22hero_id_19,22hero_id_21,22hero_id_22,22hero_id_23,22hero_id_25,22hero_id_26,22hero_id_27,22hero_id_28,22hero_id_29,22hero_id_32,22hero_id_33,22hero_id_34,22hero_id_35,22hero_id_36,22hero_id_37,22hero_id_38,22hero_id_39,22hero_id_40,22hero_id_41,22hero_id_42,22hero_id_43,22hero_id_44,22hero_id_45,22hero_id_46,22hero_id_47,22hero_id_48,22hero_id_49,22hero_id_50,22hero_id_51,22hero_id_52,22hero_id_53,22hero_id_54,22hero_id_55,22hero_id_56,22hero_id_57,22hero_id_58,22hero_id_59,22hero_id_60,22hero_id_61,22hero_id_62,22hero_id_63,22hero_id_65,22hero_id_67,22hero_id_68,22hero_id_70,22hero_id_71,22hero_id_72,22hero_id_73,22hero_id_74,22hero_id_75,22hero_id_76,22hero_id_77,22hero_id_78,22hero_id_79,22hero_id_80,22hero_id_81,22hero_id_82,22hero_id_84,22hero_id_85,22hero_id_86,22hero_id_88,22hero_id_89,22hero_id_92,22hero_id_93,22hero_id_94,22hero_id_95,22hero_id_96,22hero_id_97,22hero_id_98,22hero_id_99,22hero_id_101,22hero_id_102,22hero_id_104,22hero_id_106,22hero_id_107,22hero_id_108,22hero_id_109,22hero_id_110,22hero_id_112,22hero_id_113,22hero_id_114,22hero_id_120,22hero_id_121,22hero_id_129
match_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4870161807,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5103391298,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5062411190,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,1,0,1,1,1,0,1,1,1,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4971396775,1,1,1,0,1,1,1,0,1,1,1,0,1,1,1,0,1,0,1,1,1,1,1,0,1,1,1,0,1,0,1,1,1,0,1,1,1,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Everything seems to be in order. Let's keep going and perform the second split

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_remainder, y_remainder, test_size = 0.1, random_state = SEED) ## see top, SEED = 17

In [142]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

((940, 2088), (105, 2088), (940,), (105,))

## RFC 0 

Testing to see what raw out of the box looks like (i.e., no dummy variables)

In [228]:
from sklearn.model_selection import train_test_split
X_rtrain, X_rvalid, y_train, y_valid = train_test_split(X_rremainder, y_rremainder, test_size = 0.1, random_state = SEED) ## see top, SEED = 17

In [229]:
X_rtrain.shape, X_rvalid.shape, y_train.shape, y_valid.shape

((940, 66), (105, 66), (940,), (105,))

In [235]:
# R forest with no dummies
from sklearn.ensemble import RandomForestClassifier
modelr = RandomForestClassifier(random_state=SEED)
modelr.fit(X_rtrain, y_train)

# https://www.kaggle.com/kashnitsky/dota-2-win-prediction-random-forest-starter
y_pred_rvalid = modelr.predict(X_rvalid)
y_pred_proba_rvalid = modelr.predict_proba(X_rvalid)[:, 1]


# calcuate ROC-AUC for each split
cv_scores_rf = cross_val_score(modelr, X_rtrain, y_train, cv=5) #scoring='roc_auc'
print(cv_scores_rf)

[0.48404255 0.49468085 0.5212766  0.4787234  0.5       ]


In [237]:
# Get ROC_AUC and Accuracy
valid_score = roc_auc_score(y_valid, y_pred_rvalid)
print('Validation ROC-AUC score:', valid_score)

valid_accuracy = accuracy_score(y_valid, y_pred_proba_rvalid > 0.5)
print('Validation accuracy of P>0.5 classifier:', valid_accuracy)

Validation ROC-AUC score: 0.42078113485630064
Validation accuracy of P>0.5 classifier: 0.41904761904761906


## RFC 1. 

Testing to see what the out of the box results look like 

In [100]:
# Try this once with 10 estimators 

model1 = RandomForestClassifier(n_estimators=10, n_jobs=-1, max_depth=None, random_state=SEED)
model1.fit(X_train, y_train)

# https://www.kaggle.com/kashnitsky/dota-2-win-prediction-random-forest-starter
y_pred = model.predict_proba(X_valid)[:, 1]

In [101]:
y_pred

array([0.3, 0.4, 0.8, 0.6, 0.4, 0.4, 0.4, 0.3, 0.4, 0.7, 0.4, 0.4, 0.6,
       0.8, 0.5, 0.6, 0.9, 0.4, 0.7, 0.5, 0.2, 0.4, 0.4, 0.3, 0.7, 0.7,
       0.4, 0.7, 0.4, 0.3, 0.5, 0.7, 0.4, 0.5, 0.8, 0.3, 0.3, 0.3, 0.9,
       0.2, 0.7, 0.4, 0.7, 0.6, 0.5, 0.5, 0.4, 0.9, 0.6, 0.5, 0.9, 0.6,
       0.7, 0.5, 0.7, 0.5, 0.9, 0.3, 0.3, 0.7, 0.7, 0.7, 0.9, 0.8, 0.5,
       0.5, 0.8, 0.7, 0.5, 0.3, 0.3, 0.7, 0.5, 0.2, 0.7, 0.8, 0.1, 0.3,
       0.8, 0.7, 0.4, 0.6, 0.7, 0.5, 0.1, 0.7, 0.6, 0.8, 0.4, 0.5, 0.3,
       0.3, 0.4, 0.4, 0.3, 0.6, 0.8, 0.4, 0.9, 0.6, 0.6, 0.5, 0.4, 0.5,
       0.7])

In [102]:
valid_score = roc_auc_score(y_valid, y_pred)
print('Validation ROC-AUC score:', valid_score)

Validation ROC-AUC score: 0.3588798820928518


In [103]:
valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
print('Validation accuracy of P>0.5 classifier:', valid_accuracy)

Validation accuracy of P>0.5 classifier: 0.41904761904761906


## RFC 2.

Test with 100 estimatores

In [153]:
%%time
# Try this once with 100 estimators 

model2 = RandomForestClassifier(n_estimators=100, n_jobs=-1, max_depth=None, random_state=SEED)
model2.fit(X_train, y_train)

# https://www.kaggle.com/kashnitsky/dota-2-win-prediction-random-forest-starter
y_pred = model2.predict_proba(X_valid)[:, 1]

Wall time: 689 ms


In [154]:
y_pred

array([0.475 , 0.47  , 0.63  , 0.42  , 0.57  , 0.57  , 0.51  , 0.36  ,
       0.55  , 0.76  , 0.57  , 0.5   , 0.64  , 0.53  , 0.37  , 0.46  ,
       0.56  , 0.58  , 0.63  , 0.39  , 0.365 , 0.53  , 0.56  , 0.52  ,
       0.52  , 0.6   , 0.32  , 0.7   , 0.39  , 0.43  , 0.56  , 0.52  ,
       0.46  , 0.4425, 0.56  , 0.3   , 0.37  , 0.5   , 0.68  , 0.35  ,
       0.45  , 0.47  , 0.53  , 0.57  , 0.55  , 0.69  , 0.41  , 0.73  ,
       0.53  , 0.51  , 0.55  , 0.56  , 0.55  , 0.35  , 0.54  , 0.54  ,
       0.44  , 0.38  , 0.21  , 0.53  , 0.71  , 0.62  , 0.85  , 0.69  ,
       0.67  , 0.54  , 0.54  , 0.51  , 0.42  , 0.44  , 0.44  , 0.62  ,
       0.54  , 0.36  , 0.56  , 0.75  , 0.28  , 0.36  , 0.75  , 0.49  ,
       0.33  , 0.66  , 0.63  , 0.41  , 0.3625, 0.5   , 0.63  , 0.63  ,
       0.59  , 0.44  , 0.54  , 0.32  , 0.47  , 0.44  , 0.27  , 0.59  ,
       0.56  , 0.44  , 0.52  , 0.6   , 0.5   , 0.59  , 0.47  , 0.36  ,
       0.31  ])

In [155]:
valid_score = roc_auc_score(y_valid, y_pred)
print('Validation ROC-AUC score:', valid_score)

Validation ROC-AUC score: 0.38025055268975677


In [156]:
valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
print('Validation accuracy of P>0.5 classifier:', valid_accuracy)

Validation accuracy of P>0.5 classifier: 0.38095238095238093


## RFC Model 3

### Scaling 

Trying with experimenting with minmax and standardscaler

In [105]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler1 = MinMaxScaler()
scaler2 = StandardScaler()

# Scaled using minmax
X_train_s1 = MinMaxScaler().fit_transform(X_train)
X_valid_s1 = MinMaxScaler().fit_transform(X_valid)

# Scaled using standard scaler
X_train_s2 = StandardScaler().fit_transform(X_train)
X_valid_s2 = StandardScaler().fit_transform(X_valid)

In [121]:
scaler1 = MinMaxScaler()
scaler2 = StandardScaler()

#X = StandardScaler().fit(X).transform(X)
X_train_s1 = scaler1.fit(X_train).transform(X_train)
X_valid_s1 = scaler1.fit(X_train).transform(X_valid)
X_train_s2 = scaler2.fit(X_train).transform(X_train)
X_valid_s2 = scaler2.fit(X_train).transform(X_valid)

In [161]:
# Try this once with 10 estimators 

from sklearn.ensemble import RandomForestClassifier
model3 = RandomForestClassifier(n_estimators=10, n_jobs=-1, max_depth=None, random_state=SEED)
model3.fit(X_train_s1, y_train)

# https://www.kaggle.com/kashnitsky/dota-2-win-prediction-random-forest-starter
y_pred = model.predict_proba(X_valid_s1)[:, 1]

In [162]:
y_pred

array([0.3, 0.4, 0.8, 0.6, 0.4, 0.4, 0.4, 0.3, 0.4, 0.7, 0.4, 0.4, 0.6,
       0.8, 0.5, 0.6, 0.9, 0.4, 0.7, 0.5, 0.2, 0.4, 0.4, 0.3, 0.7, 0.7,
       0.4, 0.7, 0.4, 0.3, 0.5, 0.7, 0.4, 0.5, 0.8, 0.3, 0.3, 0.3, 0.9,
       0.2, 0.7, 0.4, 0.7, 0.6, 0.5, 0.5, 0.4, 0.9, 0.6, 0.5, 0.9, 0.6,
       0.7, 0.5, 0.7, 0.5, 0.9, 0.3, 0.3, 0.7, 0.7, 0.7, 0.9, 0.8, 0.5,
       0.5, 0.8, 0.7, 0.5, 0.3, 0.3, 0.7, 0.5, 0.2, 0.7, 0.8, 0.1, 0.3,
       0.8, 0.7, 0.4, 0.6, 0.7, 0.5, 0.1, 0.7, 0.6, 0.8, 0.4, 0.5, 0.3,
       0.3, 0.4, 0.4, 0.3, 0.6, 0.8, 0.4, 0.9, 0.6, 0.6, 0.5, 0.4, 0.5,
       0.7])

In [163]:
from sklearn.metrics import roc_auc_score

valid_score = roc_auc_score(y_valid, y_pred)
print('Validation ROC-AUC score:', valid_score)

Validation ROC-AUC score: 0.3588798820928518


In [164]:
from sklearn.metrics import accuracy_score

valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
print('Validation accuracy of P>0.5 classifier:', valid_accuracy)

Validation accuracy of P>0.5 classifier: 0.41904761904761906


## RFC Model 4. 

In [157]:
# Try this once with 10 estimators 

from sklearn.ensemble import RandomForestClassifier
model4 = RandomForestClassifier(n_estimators=10, n_jobs=-1, max_depth=None, random_state=SEED)
model4.fit(X_train_s2, y_train)

# https://www.kaggle.com/kashnitsky/dota-2-win-prediction-random-forest-starter
y_pred = model4.predict_proba(X_valid_s2)[:, 1]

In [158]:
y_pred

array([0.3, 0.4, 0.8, 0.6, 0.4, 0.4, 0.4, 0.3, 0.4, 0.7, 0.4, 0.4, 0.6,
       0.8, 0.5, 0.6, 0.9, 0.4, 0.7, 0.5, 0.2, 0.4, 0.4, 0.3, 0.7, 0.7,
       0.4, 0.7, 0.4, 0.3, 0.5, 0.7, 0.4, 0.5, 0.8, 0.3, 0.3, 0.3, 0.9,
       0.2, 0.7, 0.4, 0.7, 0.6, 0.5, 0.5, 0.4, 0.9, 0.6, 0.5, 0.9, 0.6,
       0.7, 0.5, 0.7, 0.5, 0.9, 0.3, 0.3, 0.7, 0.7, 0.7, 0.9, 0.8, 0.5,
       0.5, 0.8, 0.7, 0.5, 0.3, 0.3, 0.7, 0.5, 0.2, 0.7, 0.8, 0.1, 0.3,
       0.8, 0.7, 0.4, 0.6, 0.7, 0.5, 0.1, 0.7, 0.6, 0.8, 0.4, 0.5, 0.3,
       0.3, 0.4, 0.4, 0.3, 0.6, 0.8, 0.4, 0.9, 0.6, 0.6, 0.5, 0.4, 0.5,
       0.7])

In [159]:
valid_score = roc_auc_score(y_valid, y_pred)
print('Validation ROC-AUC score:', valid_score)

Validation ROC-AUC score: 0.3588798820928518


In [160]:
valid_accuracy = accuracy_score(y_valid, y_pred > 0.5)
print('Validation accuracy of P>0.5 classifier:', valid_accuracy)

Validation accuracy of P>0.5 classifier: 0.41904761904761906


## Cross Validation

Try with a 5 fold shuffle split. On ENTIRE training set 

In [35]:
# Encoded/Scaled train/test features

X_train1 = df_full_features_scaled[:idx_split]
X_test1 = df_full_features_scaled[idx_split:]

X_train = df_full_features_scaled[:idx_split]
X_test = df_full_features_scaled[idx_split:]

y_train = train_targets.radiant_win
y_test = test_targets.radiant_win

In [36]:
# Try shufflesplit cross val 
cv = ShuffleSplit(n_splits=5, test_size=0.1, random_state=SEED)

In [38]:
%%time
model_rf1 = RandomForestClassifier(n_estimators=100, n_jobs=-1,
                                   max_depth=None, random_state=17)

# calcuate ROC-AUC for each split
cv_scores_rf1 = cross_val_score(model_rf1, X_train, y_train, cv=cv, scoring='roc_auc', n_jobs=-1)

Wall time: 4.48 s


In [40]:
%%time

model_rf2 = RandomForestClassifier(n_estimators=100, n_jobs=-1,
                                   min_samples_leaf=3, random_state=17)

cv_scores_rf2 = cross_val_score(model_rf2, X_train, y_train, cv=cv, 
                                scoring='roc_auc', n_jobs=-1)

Wall time: 2.94 s


In [42]:
cv_scores_rf1, cv_scores_rf2

(array([0.38025055, 0.45435917, 0.43645606, 0.4247076 , 0.44448476]),
 array([0.39498895, 0.47280179, 0.41394336, 0.40643275, 0.41219158]))

In [43]:
print('Model 1 mean score:', cv_scores_rf1.mean())
print('Model 2 mean score:', cv_scores_rf2.mean())

Model 1 mean score: 0.4280516289766436
Model 2 mean score: 0.42007168404823253


In [44]:
cv_scores_rf2 > cv_scores_rf1

array([ True,  True, False, False, False])

In [23]:
# https://www.kaggle.com/vladislavkisin/dota-2-new-approaches-model-comparison-json
#Count CV scoring and houldout scoring: 
holdout_score_1 = roc_auc_score(y_valid, model.predict_proba(X_valid)[:,1]) # holdout = validation set
print('Holdout score: ', holdout_score_1)

Holdout score:  0.47310243183493006


In [ ]:
cv_score_1 = cross_val_score(model, full_df_mod[:idx_split], y_train, cv=cv, scoring = 'roc_auc') 

In [ ]:
#New results.
print('CV scores: ', cv_score_1)
print('CV mean: ', cv_score_1.mean())
print('CV std:', cv_score_1.std())
print('Holdout score: ', holdout_score_1)
print('Better results on CV: ', cv_score_1>cv_score)

## Testing out XGBoost

In [23]:
params = {}
params["objective"] =  "binary:logistic"
params["booster"] = "gbtree"
params["max_depth"] = 7
params["eval_metric"] = 'roc_auc'
params["subsample"] = 0.8
params["colsample_bytree"] = 0.8
params["silent"] = 1
params["seed"] = 4
params["eta"] = 0.1

plst = list(params.items())

In [51]:
d_train = xgb.DMatrix(X_train, label= y_train)
d_valid = xgb.DMatrix(X_valid, label= y_valid)

In [63]:
import xgboost as xgb
train_feat = xgb.DMatrix(X_train)
train_targ = xgb.DMatrix(y_valid)

valid_feat = xgb.DMatrix(X_valid)
valid_targ = xgb.DMatrix(y_valid)

ValueError: ('Expecting 2 dimensional numpy.ndarray, got: ', (105,))

In [52]:
param1 ={
    'max_depth': 7,
    #'objective': 'binary:logistic',
    'num_class': 2,
    'eta': 0.1,
    #'booster': 'gbtree',
    'n_jobs': -1,
    'random_state': 17,
    'learning_rate': 0.01,
    'eval_metric': 'roc_auc'
}
epochs=10

In [53]:
param = {
    'eta': 0.3, 
    'max_depth': 3,  
    'objective': 'binary:logistic',  
    'num_class': 2} 

steps = 20  # The number of training iterations

In [67]:
xgb_logit1 = XGBClassifier(n_jobs=-1, random_state=17, learning_rate=0.01)

In [71]:
%%time
model = xgb_logit1.fit(X_train, y_train)
print(f"XG Boost score: {xgb_logit1.score(X_valid, y_valid)}")

XG Boost score: 0.44761904761904764
Wall time: 11.7 s


In [ ]:
model = xgb_logit1.train(param, d_train, steps)
# print(f"XG Boost score: {xgb_logit1.score(X_valid, y_valid)}")

In [57]:
model = xgb.train(param, D_train, steps)

NameError: name 'D_train' is not defined

In [21]:
from xgboost import XGBClassifier 
xgb_logit = XGBClassifier(learning_rate=0.01, booster='gbtree', n_jobs=-1, random_state=SEED)

xgb_logit.fit(X_train,y_train)
print(f"XG Boost score: {xgb_logit.score(X_valid, y_valid)}")

XG Boost score: 0.47619047619047616


In [31]:
from xgboost import XGBClassifier

XGB_model = XGBClassifier(n_jobs=3, random_state=SEED)
RF_model = RandomForestClassifier(n_jobs=3, random_state=SEED)

RF_model.fit(X_train, y_train)
XGB_model.fit(X_train, y_train)

print(f"Random Forest score: {RF_model.score(X_valid, y_valid)}")
print(f"XG Boost score: {XGB_model.score(X_valid, y_valid)}")

Random Forest score: 0.38095238095238093
XG Boost score: 0.41904761904761906


In [21]:
%%time
# compare with 1000 estimators
XGB_model2 = XGBClassifier(n_estimators=1000, n_jobs=-1, random_state=SEED)
RF_model2 = RandomForestClassifier(n_estimators=1000, n_jobs=-1, random_state=SEED)

RF_model2.fit(X_train, y_train)
XGB_model2.fit(X_train, y_train)

print(f"Random Forest score: {RF_model2.score(X_valid, y_valid)}")
print(f"XG Boost score: {XGB_model2.score(X_valid, y_valid)}")

Random Forest score: 0.4
XG Boost score: 0.3904761904761905
CPU times: user 1min 38s, sys: 709 ms, total: 1min 39s
Wall time: 1min 29s


Marginal gains for random forest and loss for XGboost